In [1]:
from tensorflow.keras.utils import normalize
import os
import glob
import cv2 as cv
import numpy as np
from matplotlib import pyplot as plt
import sys
import random
import os
from keras.preprocessing import image
import tensorflow as tf 
import keras 
import cv2
import numpy as np
from matplotlib import pyplot as plt
import keras
import tensorflow as tf
np.set_printoptions(threshold=sys.maxsize)

In [2]:
!pip install --upgrade -q kaggle

!mkdir /root/.kaggle
import json
token = {
    "username": "zahidhussain909",
    "key": "39a06efd89d0f2a699143b8d3d62b216"
}

with open('/root/.kaggle/kaggle.json', 'w') as config_file:
    json.dump(token, config_file)
!chmod 600 /root/.kaggle/kaggle.json

!kaggle datasets download -d zahidhussain909/retina-segmentation-saved-model
!kaggle datasets download -d mohamedberrimi/oct-images-balanced-version
!kaggle datasets download -d zahidhussain909/best-model-oct-denoising

import zipfile
zipref=zipfile.ZipFile("/content/retina-segmentation-saved-model.zip",'r')
zipref.extractall()
zipref.close()
zipref=zipfile.ZipFile("/content/oct-images-balanced-version.zip",'r')
zipref.extractall()
zipref.close()
zipref=zipfile.ZipFile("/content/best-model-oct-denoising.zip",'r')
zipref.extractall()
zipref.close()

!rm -rf /content/retina-segmentation-saved-model.zip
!rm -rf /content/oct-images-balanced-version.zip
!rm -rf /content/best-model-oct-denoising.zip

100% 21.0M/21.0M [00:00<00:00, 51.2MB/s]
100% 21.0M/21.0M [00:00<00:00, 44.6MB/s]
100% 4.15G/4.15G [00:39<00:00, 111MB/s]
100% 4.15G/4.15G [00:39<00:00, 112MB/s]
 69% 5.00M/7.21M [00:00<00:00, 34.6MB/s]
100% 7.21M/7.21M [00:00<00:00, 46.1MB/s]


In [3]:
import tensorflow as tf
model = tf.keras.models.load_model('/content/retina_segmentation_8_layer (1).hdf5')

In [4]:
import tensorflow as tf
model_den = tf.keras.models.load_model("/content/best_model_OCT_denoising.h5")

In [15]:
test_test = []
val_image = []

orig_path = '/content/dme_seg/'

SIZE_X = 640
SIZE_Y = 640

SIZE_x = 180
SIZE_y = 180

n_classes= 9 

test_path = "/content/oct_balanced_version/Balanced/NORMAL"

val_images = sorted(os.listdir('/content/OCT_Balanced_Version/Balanced/NORMAL'))
im_base='/content/OCT_Balanced_Version/Balanced/NORMAL/'
val_images = val_images[:100]
for im in val_images:
    img_path = im_base + im

    val_image = []
    img = tf.keras.utils.load_img(img_path, target_size=(180,180), color_mode= 'grayscale')
    img = tf.keras.utils.img_to_array(img)
    img = img/255
    val_image.append(img)
    train_df = np.array(val_image)
    img = cv.imread(img_path, 0)       
    img = cv.resize(img, (SIZE_y, SIZE_x))
    pred= model_den.predict(train_df)
    pred=np.reshape(pred, (180,180))
    orig_img=img
    den_img=pred

    dst = cv.medianBlur(pred,1)
    blurred = cv.GaussianBlur(dst, (17,17), 0)
    ret,th2 = cv.threshold(blurred,0.215,1,cv.ADAPTIVE_THRESH_MEAN_C)
    th2[th2!=0] = 255
    thresh_img= np.multiply(th2,orig_img)
    
    img = cv2.resize(thresh_img, (SIZE_Y, SIZE_X))
    #test_test.append(img) 
    np.append(test_test,img)
    test_test = np.array(test_test)

    test_test = np.expand_dims(test_test, axis=3)
    test_test = normalize(test_test, axis=1)
    test1 = test_test[0]
    test_img_norm=test1[:,:,0][:,:,None]
    test=np.expand_dims(test_img_norm, 0)
    prediction = (model.predict(test))
    predicted_img = np.argmax(prediction, axis=3)[0,:,:]

    plt.figure(figsize=(10, 10))
    plt.imshow(predicted_img, cmap='jet')


1/1 [==============================] - 1s 579ms/step


AxisError: ignored

In [ ]:
test_test = []
val_image = []

orig_path = '/content/dme_seg/'

SIZE_X = 640
SIZE_Y = 640

SIZE_x = 180
SIZE_y = 180

n_classes= 9 

test_path = "/content/oct_balanced_version/Balanced/NORMAL"

val_images = sorted(os.listdir('/content/OCT_Balanced_Version/Balanced/NORMAL'))
im_base='/content/OCT_Balanced_Version/Balanced/NORMAL/'
val_images = val_images[:100]
for im in val_images:
    img_path = im_base + im

    val_image = []
    img = tf.keras.utils.load_img(img_path, target_size=(180,180), color_mode= 'grayscale')
    img = tf.keras.utils.img_to_array(img)
    img = img/255
    val_image.append(img)
    train_df = np.array(val_image)
    img = cv.imread(img_path, 0)       
    img = cv.resize(img, (SIZE_y, SIZE_x))
    pred= model_den.predict(train_df)
    pred=np.reshape(pred, (180,180))
    orig_img=img
    den_img=pred

    dst = cv.medianBlur(pred,1)
    blurred = cv.GaussianBlur(dst, (17,17), 0)
    ret,th2 = cv.threshold(blurred,0.215,1,cv.ADAPTIVE_THRESH_MEAN_C)
    th2[th2!=0] = 255
    thresh_img= np.multiply(th2,orig_img)

    img = cv2.resize(thresh_img, (SIZE_Y, SIZE_X))
    test_test = np.array(img)
    #test_test = np.expand_dims(test_test, axis=3)
    test_test = normalize(test_test, axis=1)
    test1 = test_test[0]
    test_img_norm=test1[:,:,0][:,:,None]
    test=np.expand_dims(test_img_norm, 0)
    prediction = (model.predict(test))
    predicted_img = np.argmax(prediction, axis=3)[0,:,:]

    plt.figure(figsize=(10, 10))
    plt.imshow(predicted_img, cmap='jet')
